# Common imports & library functions

In [52]:
import doctest
import itertools
import math

# Day 1: The Tyranny of the Rocket Equation

In [5]:
def fuel_requirements(mass):
    """
    >>> fuel_requirements(12)
    2
    >>> fuel_requirements(14)
    2
    >>> fuel_requirements(1969)
    654
    >>> fuel_requirements(100756)
    33583
    """
    return mass // 3 - 2

def meta_fuel_requirements(mass):
    """
    >>> meta_fuel_requirements(14)
    2
    >>> meta_fuel_requirements(1969)
    966
    >>> meta_fuel_requirements(100756)
    50346
    """
    if (req := fuel_requirements(mass)) < 0:
        return 0
    else:
        return req + meta_fuel_requirements(req)


def total_fuel_requirements(masses):
    return sum(fuel_requirements(m) for m in masses)

def total_meta_fuel_requirements(masses):
    return sum(meta_fuel_requirements(m) for m in masses)

In [6]:
# Run unit tests
doctest.run_docstring_examples(fuel_requirements, globs=None, verbose=True)

Finding tests in NoName
Trying:
    fuel_requirements(12)
Expecting:
    2
ok
Trying:
    fuel_requirements(14)
Expecting:
    2
ok
Trying:
    fuel_requirements(1969)
Expecting:
    654
ok
Trying:
    fuel_requirements(100756)
Expecting:
    33583
ok


In [7]:
# Run unit tests
doctest.run_docstring_examples(meta_fuel_requirements, globs=None, verbose=True)

Finding tests in NoName
Trying:
    meta_fuel_requirements(14)
Expecting:
    2
ok
Trying:
    meta_fuel_requirements(1969)
Expecting:
    966
ok
Trying:
    meta_fuel_requirements(100756)
Expecting:
    50346
ok


In [8]:
# Final answers
with open('day1_input.txt') as f:
    values = [int(l.strip()) for l in f]
    print('Part 1: ', total_fuel_requirements(values))
    print('Part 2: ', total_meta_fuel_requirements(values))

Part 1:  3152375
Part 2:  4725720


# Day 2: 1202 Program Alarm

In [88]:
class Opcode:
    ADD = 1
    MULT = 2
    HALT = 99

def execute(program, ip=0):
    """
    >>> execute([1,0,0,0,99])
    [2, 0, 0, 0, 99]
    >>> execute([1,1,1,4,99,5,6,0,99])
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
    """
    memory = program[:]
    while ip < len(memory):
        if memory[ip] == Opcode.HALT:
            return memory
        
        src_a, src_b, dst = memory[ip+1:ip+4]
        if memory[ip] == Opcode.ADD:
            memory[dst] = memory[src_a] + memory[src_b]
        elif memory[ip] == Opcode.MULT:
            memory[dst] = memory[src_a] * memory[src_b]
        else:
            raise Exception('Invalid opcode: {}'.format(memory[ip]))
        ip += 4
    raise Exception('Execution terminated without halt instruction')
    
def find_noun_verb_pair(program, expected_output):
    for (noun, verb) in itertools.product(range(100), range(100)):
        program[1] = noun
        program[2] = verb
        if execute(program)[0] == expected_output:
            return '{:02}{:02}'.format(noun, verb)

In [89]:
# Run unit tests
doctest.run_docstring_examples(execute, globs=None, verbose=True)

Finding tests in NoName
Trying:
    execute([1,0,0,0,99])
Expecting:
    [2, 0, 0, 0, 99]
ok
Trying:
    execute([1,1,1,4,99,5,6,0,99])
Expecting:
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
ok


In [93]:
# Run integration test
with open('day2_input.txt') as f:
    values = [int(l) for l in f.read().split(',')]
    actual = find_noun_verb_pair(values, 5110675)
    if actual != '1202':
        print('Failed! Got {}'.format(actual))
    else:
        print('Passed!')

Passed!


In [95]:
# Final answers
with open('day2_input.txt') as f:
    values = [int(l) for l in f.read().split(',')]
    # Restore to 1202 state:
    values[1] = 12
    values[2] = 2
    print('Part 1: ', execute(values))
    print('Part 2: ', find_noun_verb_pair(values, 19690720))

Part 1:  [5110675, 12, 2, 2, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 9, 1, 36, 1, 5, 19, 37, 2, 9, 23, 111, 1, 27, 5, 112, 2, 31, 13, 560, 1, 35, 9, 563, 1, 39, 10, 567, 2, 43, 9, 1701, 1, 47, 5, 1702, 2, 13, 51, 8510, 1, 9, 55, 8513, 1, 5, 59, 8514, 2, 6, 63, 17028, 1, 5, 67, 17029, 1, 6, 71, 17031, 2, 9, 75, 51093, 1, 79, 13, 51098, 1, 83, 13, 51103, 1, 87, 5, 51104, 1, 6, 91, 51106, 2, 95, 13, 255530, 2, 13, 99, 1277650, 1, 5, 103, 1277651, 1, 107, 10, 1277655, 1, 111, 13, 1277660, 1, 10, 115, 1277664, 1, 9, 119, 1277667, 2, 6, 123, 2555334, 1, 5, 127, 2555335, 2, 6, 131, 5110670, 1, 135, 2, 5110672, 1, 139, 9, 0, 99, 2, 14, 0, 0]
Part 2:  4847


# Day 3: Crossed Wires

In [127]:
DELTAS = {
    'R': (1, 0),
    'L': (-1, 0),
    'U': (0, 1),
    'D': (0, -1),
}
ORIGIN = (0, 0)

def move(pos, direction):
    return (pos[0] + DELTAS[direction][0],
            pos[1] + DELTAS[direction][1])

def dist(pos):
    return sum(abs(c) for c in pos)

def segment_coordinates(pos, segment):
    """
    >>> segment_coordinates((0, 0), 'R2')
    [(1, 0), (2, 0)]
    >>> segment_coordinates((-1, 2), 'D2')
    [(-1, 1), (-1, 0)]
    """
    coords = []
    direction, steps = segment[0], int(segment[1:])
    for _ in range(steps):
        coords.append(move(pos, direction))
        pos = coords[-1]
    return coords

def wire_coordinates(wire_segments):
    """
    >>> wire_coordinates(['R2', 'U2'])
    [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)]
    >>> wire_coordinates(['R1', 'U1', 'L1', 'D1'])
    [(0, 0), (1, 0), (1, 1), (0, 1), (0, 0)]
    """
    pos = (0, 0)
    coords = [pos]
    for segment in wire_segments:
        coords.extend(segment_coordinates(pos, segment))
        pos = coords[-1]
    return coords

def closest_crossing_naive(wire1, wire2):
    """
    >>> wire1 = ['R75','D30','R83','U83','L12','D49','R71','U7','L72']
    >>> wire2 = ['U62','R66','U55','R34','D71','R55','D58','R83']
    >>> closest_crossing_naive(wire1, wire2)
    159
    >>>
    >>> wire1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'.split(',')
    >>> wire2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'.split(',')
    >>> closest_crossing_naive(wire1, wire2)
    135
    """
    coords1 = set(wire_coordinates(wire1))
    coords2 = set(wire_coordinates(wire2))
    coords1.remove(ORIGIN)
    coords2.remove(ORIGIN)
    return dist(min(coords1 & coords2, key=dist))

def least_delay_crossing(wire1, wire2):
    """
    >>> wire1 = ['R75','D30','R83','U83','L12','D49','R71','U7','L72']
    >>> wire2 = ['U62','R66','U55','R34','D71','R55','D58','R83']
    >>> least_delay_crossing(wire1, wire2)
    610
    >>>
    >>> wire1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'.split(',')
    >>> wire2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'.split(',')
    >>> least_delay_crossing(wire1, wire2)
    410
    """
    delays1 = {coord: delay for (delay, coord) in enumerate(wire_coordinates(wire1))}
    delays2 = {coord: delay for (delay, coord) in enumerate(wire_coordinates(wire2))}
    del delays1[ORIGIN]
    del delays2[ORIGIN]
    delay_fn = lambda coord: delays1[coord] + delays2[coord]
    return delay_fn(min(set(delays1.keys()) & set(delays2.keys()), key=delay_fn))

In [128]:
# Run unit tests
doctest.run_docstring_examples(segment_coordinates, globs=None, verbose=True)
doctest.run_docstring_examples(wire_coordinates, globs=None, verbose=True)
doctest.run_docstring_examples(closest_crossing_naive, globs=None, verbose=True)
doctest.run_docstring_examples(least_delay_crossing, globs=None, verbose=True)

Finding tests in NoName
Trying:
    segment_coordinates((0, 0), 'R2')
Expecting:
    [(1, 0), (2, 0)]
ok
Trying:
    segment_coordinates((-1, 2), 'D2')
Expecting:
    [(-1, 1), (-1, 0)]
ok
Finding tests in NoName
Trying:
    wire_coordinates(['R2', 'U2'])
Expecting:
    [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)]
ok
Trying:
    wire_coordinates(['R1', 'U1', 'L1', 'D1'])
Expecting:
    [(0, 0), (1, 0), (1, 1), (0, 1), (0, 0)]
ok
Finding tests in NoName
Trying:
    wire1 = ['R75','D30','R83','U83','L12','D49','R71','U7','L72']
Expecting nothing
ok
Trying:
    wire2 = ['U62','R66','U55','R34','D71','R55','D58','R83']
Expecting nothing
ok
Trying:
    closest_crossing_naive(wire1, wire2)
Expecting:
    159
ok
Trying:
    wire1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'.split(',')
Expecting nothing
ok
Trying:
    wire2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'.split(',')
Expecting nothing
ok
Trying:
    closest_crossing_naive(wire1, wire2)
Expecting:
    135
ok
Finding tests in NoName
Tryin

In [129]:
# Final answers
with open('day3_input.txt') as f:
    wires = f.readlines()
    wire1 = wires[0].split(',')
    wire2 = wires[1].split(',')
    print('Part 1: ', closest_crossing_naive(wire1, wire2))
    print('Part 2: ', least_delay_crossing(wire1, wire2))

Part 1:  308
Part 2:  12934


# Day 4: Secure Container

In [196]:
from collections import Counter

def password_increment(digits):
    """
    >>> password_increment('13')
    '14'
    >>> password_increment('399')
    '444'
    >>> password_increment('999')
    '1111'
    """
    digits = str(int(digits) + 1)
    if "0" not in digits:
        return digits
    pos = digits.index("0")
    pad = len(digits) - pos
    return digits[:pos] + digits[pos-1] * pad

def is_increasing(digits):
    """
    >>> is_increasing('123')
    True
    >>> is_increasing('231')
    False
    """
    return sorted(digits) == list(digits)

def has_repeat(digits):
    """
    >>> has_repeat('122')
    True
    >>> has_repeat('123')
    False
    >>> has_repeat('333')
    True
    >>> has_repeat('3333')
    True
    """
    return any(c > 1 for c in Counter(digits).values())

def has_even_repeat(digits):
    """
    >>> has_even_repeat('122')
    True
    >>> has_even_repeat('123')
    False
    >>> has_even_repeat('33311')
    True
    >>> has_even_repeat('333311')
    True
    """
    return any(c == 2 for c in Counter(digits).values())

def is_valid1(digits):
    """
    >>> is_valid1('123')
    False
    >>> is_valid1('1122')
    True
    >>> is_valid1('111')
    True
    """
    return is_increasing(digits) and has_repeat(digits)

def is_valid2(digits):
    """
    >>> is_valid2('123')
    False
    >>> is_valid2('1122')
    True
    >>> is_valid2('111')
    False
    """
    return is_increasing(digits) and has_even_repeat(digits)

def next_valid_password(digits, filter=is_valid1):
    """
    >>> next_valid_password('111')
    '111'
    >>> next_valid_password('911')
    '999'
    """
    while not filter(digits):
        digits = str(int(digits) + 1)
    return digits

def password_generator(start, end, filter=is_valid1):
    """
    >>> a = set(password_generator('11', '99'))
    >>> '44' in a
    True
    >>> '94' in a
    False
    """
    password = next_valid_password(start, filter)
    while int(password) <= int(end):
        if filter(password): yield password
        password = password_increment(password)

def num_passwords(start, end, filter=is_valid1):
    count = 0
    for _ in password_generator(start, end, filter=filter):
        count += 1
    return count

In [197]:
# Run unit tests
doctest.run_docstring_examples(password_increment, globs=None, verbose=True)
doctest.run_docstring_examples(is_increasing, globs=None, verbose=True)
doctest.run_docstring_examples(next_valid_password, globs=None, verbose=True)
doctest.run_docstring_examples(has_repeat, globs=None, verbose=True)
doctest.run_docstring_examples(has_even_repeat, globs=None, verbose=True)
doctest.run_docstring_examples(is_valid1, globs=None, verbose=True)
doctest.run_docstring_examples(is_valid2, globs=None, verbose=True)
doctest.run_docstring_examples(password_generator, globs=None, verbose=True)

Finding tests in NoName
Trying:
    password_increment('13')
Expecting:
    '14'
ok
Trying:
    password_increment('399')
Expecting:
    '444'
ok
Trying:
    password_increment('999')
Expecting:
    '1111'
ok
Finding tests in NoName
Trying:
    is_increasing('123')
Expecting:
    True
ok
Trying:
    is_increasing('231')
Expecting:
    False
ok
Finding tests in NoName
Trying:
    next_valid_password('111')
Expecting:
    '111'
ok
Trying:
    next_valid_password('911')
Expecting:
    '999'
ok
Finding tests in NoName
Trying:
    has_repeat('122')
Expecting:
    True
ok
Trying:
    has_repeat('123')
Expecting:
    False
ok
Trying:
    has_repeat('333')
Expecting:
    True
ok
Trying:
    has_repeat('3333')
Expecting:
    True
ok
Finding tests in NoName
Trying:
    has_even_repeat('122')
Expecting:
    True
ok
Trying:
    has_even_repeat('123')
Expecting:
    False
ok
Trying:
    has_even_repeat('33311')
Expecting:
    True
ok
Trying:
    has_even_repeat('333311')
Expecting:
    True
ok
Find

In [199]:
# Final answers
print('Part 1: ', num_passwords('172930', '683082', filter=is_valid1))
print('Part 2: ', num_passwords('172930', '683082', filter=is_valid2))

Part 1:  1675
Part 2:  1142


# Day 5: Sunny with a Chance of Asteroids

In [321]:
# Moved to intcode.py
from intcode import *

In [322]:
# Run unit tests
doctest.run_docstring_examples(parse, globs=None, verbose=True)
doctest.run_docstring_examples(execute, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse(2)
Expecting:
    (2, 0, 0, 0)
ok
Trying:
    parse(1002)
Expecting:
    (2, 0, 1, 0)
ok
Trying:
    parse(11199)
Expecting:
    (99, 1, 1, 1)
ok
Finding tests in NoName
Trying:
    execute([1,0,0,0,99])
Expecting:
    [2, 0, 0, 0, 99]
ok
Trying:
    execute([1,1,1,4,99,5,6,0,99])
Expecting:
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
ok
Trying:
    execute([1002,4,3,4,33])
Expecting:
    [1002, 4, 3, 4, 99]
ok


In [323]:
# Integration test:
# The above example program uses an input instruction to ask for a single number. 
# The program will then output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8,
# or output 1001 if the input value is greater than 8.
program = [
    3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
    1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
    999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
]
memory = execute(program)

I> 10
O> 1001


In [317]:
# Final answers
with open('day5_input.txt') as f:
    program = parse_program(f.read())
    print('Part 1: (input 1)')
    execute(program)
    print('Part 2: (input 5)')
    execute(program)

Part 1: (input 1)
1
> 0
> 0
> 0
> 0
> 0
> 0
> 0
> 0
> 0
> 6761139
Part 2: (input 5)
5
> 9217546
